**A/B Hypothesis Testing**



**Business Question **


1	Does the shipping mode have a significant impact on the average order quantity?



In [43]:
#importing the packages needed.

import numpy as np
import pandas  as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
raw=pd.read_csv("C:\\Users\DELL-PC\\Documents\\Database for Sql\\Datasets for Projects\\Raw Data\\superstore-orders.csv",
               encoding='ANSI')
raw.head(5)

,order_id,order_date,ship_date,ship_mode,customer_name,segment,state,country,market,region,...,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,year
0,AG-2011-2040,1/1/2011,6/1/2011,Standard Class,Toby Braunhardt,Consumer,Constantine,Algeria,Africa,Africa,...,Office Supplies,Storage,"Tenex Lockers, Blue",408,2,0.0,106.140,35.46,Medium,2011
1,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Office Supplies,Supplies,"Acme Trimmer, High Speed",120,3,0.1,36.036,9.72,Medium,2011
2,HU-2011-1220,1/1/2011,5/1/2011,Second Class,Annie Thurman,Consumer,Budapest,Hungary,EMEA,EMEA,...,Office Supplies,Storage,"Tenex Box, Single Width",66,4,0.0,29.640,8.17,High,2011
3,IT-2011-3647632,1/1/2011,5/1/2011,Second Class,Eugene Moren,Home Office,Stockholm,Sweden,EU,North,...,Office Supplies,Paper,"Enermax Note Cards, Premium",45,3,0.5,-26.055,4.82,High,2011
4,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",114,5,0.1,37.770,4.70,Medium,2011


In [3]:
#info of the dataset
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   order_id        51290 non-null  object 
 1   order_date      51290 non-null  object 
 2   ship_date       51290 non-null  object 
 3   ship_mode       51290 non-null  object 
 4   customer_name   51290 non-null  object 
 5   segment         51290 non-null  object 
 6   state           51290 non-null  object 
 7   country         51290 non-null  object 
 8   market          51290 non-null  object 
 9   region          51290 non-null  object 
 10  product_id      51290 non-null  object 
 11  category        51290 non-null  object 
 12  sub_category    51290 non-null  object 
 13  product_name    51290 non-null  object 
 14  sales           51290 non-null  object 
 15  quantity        51290 non-null  int64  
 16  discount        51290 non-null  float64
 17  profit          51290 non-null 

Data Cleaning and Preprocessing

In [4]:
#check for  duplicates
raw[raw.duplicated()]

,order_id,order_date,ship_date,ship_mode,customer_name,segment,state,country,market,region,...,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,year


In [5]:
#check for nulls
raw.isnull().sum()

order_id          0
order_date        0
ship_date         0
ship_mode         0
customer_name     0
segment           0
state             0
country           0
market            0
region            0
product_id        0
category          0
sub_category      0
product_name      0
sales             0
quantity          0
discount          0
profit            0
shipping_cost     0
order_priority    0
year              0
dtype: int64

Transforming Incorrect Data Types

In [6]:
# Remove commas from 'sales' column
raw['sales'] = raw['sales'].str.replace(',', '')

# Convert 'sales' column to integers
raw['sales'] = raw['sales'].astype(int)


In [7]:
#the data type of the sales column is now an integer.
raw['sales'].dtype

dtype('int32')

In [8]:
#create a uniform separator for  the order date column
raw['order_date']=raw['order_date'].str.replace('/','-')

In [9]:
#create a uniform separator for the ship date column
raw['ship_date']=raw['ship_date'].str.replace('/','-')

In [10]:
#order date formats in both d/m/Y and Y/m/d formats
raw['order_date']

0          1-1-2011
1          1-1-2011
2          1-1-2011
3          1-1-2011
4          1-1-2011
            ...    
51285    31-12-2014
51286    31-12-2014
51287    31-12-2014
51288    31-12-2014
51289    31-12-2014
Name: order_date, Length: 51290, dtype: object

In [11]:
#convert order date column from string to datetime
raw['order_date']=pd.to_datetime(raw['order_date'], format='%d-%m-%Y')

In [12]:
#convert ship date column from string to datetime
raw['ship_date']=pd.to_datetime(raw['ship_date'], errors='coerce')

In [13]:
raw['order_date'].dtype

dtype('<M8[ns]')

**EDA**

In [14]:
#Basic descriptive statistics of the dataset
raw.describe()

,order_date,ship_date,sales,quantity,discount,profit,shipping_cost,year
count,51290,19834,51290.000000,51290.000000,51290.000000,51290.000000,51290.000000,51290.000000
mean,2013-05-11 21:26:49.155780864,2013-04-14 00:24:19.312292096,246.498440,3.476545,0.142908,28.641740,26.375915,2012.777208
min,2011-01-01 00:00:00,2011-01-02 00:00:00,0.000000,1.000000,0.000000,-6599.978000,0.000000,2011.000000
25%,2012-06-19 00:00:00,2012-05-09 00:00:00,31.000000,2.000000,0.000000,0.000000,2.610000,2012.000000
50%,2013-07-08 00:00:00,2013-06-05 00:00:00,85.000000,3.000000,0.000000,9.240000,7.790000,2013.000000
75%,2014-05-22 00:00:00,2014-04-08 00:00:00,251.000000,5.000000,0.200000,36.810000,24.450000,2014.000000
max,2014-12-31 00:00:00,2015-07-01 00:00:00,22638.000000,14.000000,0.850000,8399.976000,933.570000,2014.000000
std,NaN,NaN,487.567175,2.278766,0.212280,174.424113,57.296804,1.098931


In [15]:
#getting the number uniuque values
raw.nunique()

order_id          25035
order_date         1430
ship_date           580
ship_mode             4
customer_name       795
segment               3
state              1094
country             147
market                7
region               13
product_id        10292
category              3
sub_category         17
product_name       3788
sales              2246
quantity             14
discount             27
profit            24575
shipping_cost     10037
order_priority        4
year                  4
dtype: int64

In [31]:
raw.shape

(51290, 23)

**Variable Selection**

1  Does the shipping mode have a significant impact on the average order quantity?

For this we are interested in the Mode of shipping and the order qty


In [34]:
#subseting the dataset


#Question : shipping mode vs Quantity
std_class=raw[raw['ship_mode']=='Standard Class']['quantity']
first_class=raw[raw['ship_mode']=='First Class']['quantity']
second_class=raw[raw['ship_mode']=='Second Class']['quantity']
same_day=raw[raw['ship_mode']=='Same Day']['quantity']

In [21]:
Significance_level=0.05

In [35]:
#check for the sample sizes for each of the subsets to ensure fairness
sample_size_std_class = len(std_class)
sample_size_first_class = len(first_class)
sample_size_second_class = len(second_class)
sample_size_same_day = len(same_day)

print("Sample Size - Standard Class:", sample_size_std_class)
print("Sample Size - First Class:", sample_size_first_class)
print("Sample Size - Second Class:", sample_size_second_class)
print("Sample Size - Same Day:", sample_size_same_day)


Sample Size - Standard Class: 30775
Sample Size - First Class: 7505
Sample Size - Second Class: 10309
Sample Size - Same Day: 2701


In [46]:
#caluclating the average order quantity of shiping modes
std_class_mean=std_class_sample.mean()
first_class_mean=first_class_sample.mean()
second_class_mean=second_class_sample.mean()
same_day_mean=same_day_sample.mean()


print('shipping mode mean', std_class_mean,first_class_mean,second_class_mean, same_day_mean)

shipping mode mean 3.631 3.561 3.548 3.448


**Hypothesis Definition**

Hypothesis Testing for **Standard Class**

H0: There is no significant impact between standard class shipping mode and order quantity
    
H1: There is a significant diffeence between standrad class shipping mode and order quantity

Hypothesis Testing for **First Class**

H0: There is no significant impact between First class shipping mode and order quantity
    
H1: There is a significant diffeence between First class shipping mode and order quantity

Hypothesis Testing for **Second Class**

H0: There is no significant impact between Second class shipping mode and order quantity
    
H1: There is a significant diffeence between Second class shipping mode and order quantity

Hypothesis Testing for **Same Day**

H0: There is no significant impact between same day shipping mode and order quantity
    
H1: There is a significant diffeence between same day shipping mode and order quantity

In [45]:

# t-test for standard class and first class
t_stat, p_value = stats.ttest_ind(std_class, first_class, equal_var=False)

print("T-statistic:", t_stat)
print("P-value:", p_value)

T-statistic: 0.6056396206637537
P-value: 0.5447660140814421


**Interpretation**

Interpretation:
Since the p-value (0.5448) is greater than the significance level of 5% or 0.05, we fail to reject the null hypothesis. This means that there is not enough evidence to conclude that there is a significant difference in the average order quantities between Standard Class and First Class shipping modes.

In other words, based on the sample data, we do not have sufficient evidence to suggest that the choice of shipping mode (Standard Class vs. First Class) has a significant impact on the average order quantity. That is, it is merely by chance.

In [48]:
# t-test for standard class and first class
t_stat, p_value = stats.ttest_ind(second_class, same_day, equal_var=False)

print("T-statistic:", t_stat)
print("P-value:", p_value)

T-statistic: 0.9904815564369654
P-value: 0.3219943551639985


**Interpretation**

Interpretation: Since the p-value (0.3219) is greater than the significance level of 5% or 0.05, we fail to reject the null hypothesis. This means that there is no statistically significant evidence to conclude that there is a significant difference in the average order quantities between Second Class and Same Day shipping modes.

In other words, based on the sample data, we do not have sufficient evidence to suggest that the choice of shipping mode (Second Class vs Same Day) has a significant impact on the average order quantity. That is, it is merely by chance.

**Business Insights**

From a statistical standpoint, the choice of shipping mode (Standard Class or First Class), (second class vs same day) does not appear to have a significant impact on the average order quantity. Therefore, other factors such as pricing, product selection, customer service, or marketing efforts may have a more significant influence on order quantities.

Additionally, further analysis and consideration of other factors are recommended to gain a comprehensive understanding of the relationships between shipping modes and order quantities